In [1]:
import collections
import typing

import nltk
import pandas as pd

In [2]:
DATASET: str = '../data/dataset.parquet'

In [3]:
dataset: pd.DataFrame = pd.read_parquet(DATASET)
dataset

,model,response
0,mixtral:8x7b,"""In Deutschland schätzen wir den Schutz unser..."
1,qwen:72b,""" Datenprivatheit - mehr als nur ein Trend. Un..."
2,qwen:72b,"""Datenschutz ist mehr als ein trendiges Wort -..."
3,qwen:72b,"""Data privacy matters! 🌐🔒 Waking up to yet ano..."
4,qwen:72b,"""Datenschutz oder Überwachung? Our personal in..."
...,...,...
1020,mixtral:8x7b,"""In Deutschland schätzen wir den Schutz unser..."
1021,falcon:40b,"I am sorry but as an AI language model, I cann..."
1022,llama2:70b,"\n""Datenschutz ist wichtig! 🤖🔒 Es ist Zeit, da..."
1023,falcon:40b,"As an AI language model, I do not have persona..."


In [74]:
dataset['model'].value_counts()

model
llama2:70b      267
falcon:40b      263
qwen:72b        261
mixtral:8x7b    234
Name: count, dtype: int64

In [80]:
token_count: typing.Dict[str, pd.Series] = {}

for model, group in dataset.groupby('model', observed=False):
    token: typing.List[str] = (
        ' '.join(group.response.tolist())
        .lower()
        .replace('"', '')
        .replace('#', '')
        .split()
    )
    counter = collections.Counter(token)

    for token in [*nltk.corpus.stopwords.words('english'), *nltk.corpus.stopwords.words('german')]:
        del counter[token]

    token_count[model] = pd.Series(counter, name='count').sort_values(ascending=False)

In [102]:
collated_token_count: pd.DataFrame = pd.concat(token_count, axis='columns')
collated_token_count

,falcon:40b,llama2:70b,mixtral:8x7b,qwen:72b
personal,794.0,188.0,228.0,36.0
data,752.0,263.0,97.0,85.0
information,706.0,100.0,99.0,15.0
privacy,582.0,190.0,95.0,80.0
social,544.0,6.0,6.0,2.0
...,...,...,...,...
vs.,NaN,NaN,NaN,1.0
wert,NaN,NaN,NaN,1.0
legst,NaN,NaN,NaN,1.0
persönlich,NaN,NaN,NaN,1.0


In [103]:
collated_token_count.describe()

,falcon:40b,llama2:70b,mixtral:8x7b,qwen:72b
count,3284.000000,1395.000000,559.000000,911.000000
mean,7.874543,5.556272,12.023256,3.364435
std,33.934243,19.687886,31.436079,10.104020
min,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000
50%,2.000000,1.000000,2.000000,1.000000
75%,4.000000,3.000000,6.000000,2.000000
max,794.000000,340.000000,267.000000,200.000000


In [101]:
collated_token_count.filter(
    items=[
        'privatsphäre',
        'regierungen',
        'grundrecht',
        'datenschutzgesetze',
        'schützen',
        'zustimmung',
        'privatleben',
    ],
    axis=0
)

,falcon:40b,llama2:70b,mixtral:8x7b,qwen:72b
privatsphäre,17.0,287.0,67.0,44.0
regierungen,1.0,37.0,26.0,1.0
grundrecht,NaN,NaN,5.0,NaN
datenschutzgesetze,NaN,4.0,8.0,1.0
schützen,7.0,77.0,36.0,16.0
zustimmung,1.0,18.0,6.0,NaN
privatleben,NaN,2.0,5.0,NaN
schutzmaßnahmen,NaN,NaN,NaN,5.0
